In [ ]:
import math
from logging import debug

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import copy
from math import sqrt,ceil

from PIL.ImageColor import colormap
from scipy.stats import linregress
import libraries as lib
import re
import Weer_data_inladen
from libraries import plot_trajectory_map

In [1]:
#20240713_Huy-Limburg/0026 - 20240713T080342 14h37m16s 145.044km Durnal - Heel
#Configuratie: 
#Roze roer 
#Motortype: 30100/13 (NR 6) 
#@i=7 (NR E) 
#Schakeling: Driehoek 
#Schroef: Incapa Blad 300 N, 15 
#Piloot: 
#Teun van den Elzen (72kg) (11:0
#Tristan Ploeger (80kg) (13:30-2
#Tomas Kramer (70kg) (20:00- 

#20240709_Anse_stroomafwaarts/0002 - 20240709T070429 8h16m21s 51.701km Trevoux - Givors
#Configuratie: 
#Roze roer 
#Motortype: 30100/13 (NR 6) 
#@i=7 (NR E) 
#Schakeling: Driehoek 
#Schroef: Schroef Fee (Chopped Tips) 
#Folder: 
#Folders zijn weer begonnen vanaf mapje 1 vanwege het vervangen van de sd-kaart. 
#0002: Gaat het water in <--- 
#0003: Na opnieuw opstarten. 
#Piloot: 
#Tristan Ploeger (80kg) 
#Jonathan Meuleman (70kg) 

#20240708_Valence/0231 - 20240708T111346 8h9m54s 47.648km La Voulte-sur-Rhone - Tournon-sur-Rhone
#Configuratie: 
#Roze roer 
#Motortype: 30100/13 (NR 6) 
#@i=7 (NR E) 
#Schakeling: Driehoek 
#Schroef: Schroef Fee (Chopped Tips) 
#Piloot: 
#Ruben Jans (77kg) 
#Bastiaan Buurman (75kg) 

#20240714_Limburg-Nijmegen\0028 - 20240714T061924 15h2m29s 148.594km Bunde - Lent
#Configuratie: 
#Roze roer 
#Motortype: 30100/13 (NR 6) 
#@i=7 (NR E) 
#Schakeling: Driehoek 
#Schroef: Incapa Blad 300 N, 15 m/s 
#Piloot: 
#Felix van Beusekom (78kg) (10:20-14:40) 
#Dennis van den Berg (73kg) (14:40-~17:30) 
#Jonathan Gessel (76kg) (~17:30-21:20) 

In [3]:
directory1 = "../../OneDrive - HvA/Logfiles/SOLAR/2024/20240707_20240715_terugvaart/20240713_Huy-Limburg/0026 - 20240713T080342 14h37m16s 145.044km Durnal - Heel"
directory2 = '../../OneDrive - HvA/Logfiles/SOLAR/2024/20240707_20240715_terugvaart/20240714_Limburg-Nijmegen/0028 - 20240714T061924 15h2m29s 148.594km Bunde - Lent'
directory3 = '../../OneDrive - HvA/Logfiles/SOLAR/2024/20240707_20240715_terugvaart/20240708_Valence/0231 - 20240708T111346 8h9m54s 47.648km La Voulte-sur-Rhone - Tournon-sur-Rhone'
directory4 = '../../OneDrive - HvA/Logfiles/SOLAR/2024/20240707_20240715_terugvaart/20240709_Anse_stroomafwaarts/0002 - 20240709T070429 8h16m21s 51.701km Trevoux - Givors'

result1 = lib.DataUitzoekenGui(directory1)
result2 = lib.DataUitzoekenGui(directory2)
result3 = lib.DataUitzoekenGui(directory3)
result4 = lib.DataUitzoekenGui(directory4)

SelectMultiple(description='Bestanden:', options=('0_Bus.csv', '2_MPPT_0_05_05.csv', '3_MPPT_1_05_07.csv', '4_…

Button(button_style='success', description='Bestanden Laden', icon='check', style=ButtonStyle(), tooltip='Klik…

Label(value='')

SelectMultiple(description='Bestanden:', options=('0_Bus.csv', '2_MPPT_0_05_05.csv', '3_MPPT_1_05_07.csv', '4_…

Button(button_style='success', description='Bestanden Laden', icon='check', style=ButtonStyle(), tooltip='Klik…

...

Button(button_style='success', description='Bestanden Laden', icon='check', style=ButtonStyle(), tooltip='Klik…

Label(value='')

SelectMultiple(description='Bestanden:', options=('0_Bus.csv', '2_MPPT_0_05_07.csv', '3_MPPT_1_05_07.csv', '4_…

Button(button_style='success', description='Bestanden Laden', icon='check', style=ButtonStyle(), tooltip='Klik…

Label(value='')

In [29]:
# Na het laden van de bestanden en het klikken op de knop
# Controleer of 'merged_df' in result zit
if 'merged_df' in result1:
    merged_df1 = result1['merged_df'].copy()
    #merged_df2 = result2['merged_df'].copy()
    #merged_df3 = result3['merged_df'].copy()
    #merged_df4 = result4['merged_df'].copy()
    # Gebruik merged_df zoals je wilt
else:
    print("De data is nog niet geladen.")

In [30]:
merged_df1

,1_Datalogger port,"1_Dataloggertijd, in s","1_MPPT-tijd, in s","1_Paneel 1 ingangsspanning, in V","1_Paneel 1 ingangsstroom, in I","1_Paneel 1 instantaan ingangsvermogen, in W","1_Paneel 1 totale ingangsenergie, in J","1_Paneel 2 ingangsspanning, in V","1_Paneel 2 ingangsstroom, in I","1_Paneel 2 instantaan ingangsvermogen, in W",...,"5_Gps-latitude, in graden",5_GPS richting,"5_Gps-tijd, in s",5_Format header (>05|05),"5_Paneel 1 ingangsspanning, in V","5_Uitgangsspanning, in V","5_Kanaal 1 instantaan uitgangsvermogen, in W","5_Gps-snelheid, in km/u",5_Pirano 1,5_Pirano 2
Indextijd,,,,,,,,,,,,,,,,,,,,,
0 days 00:02:17,2.0,137.432109,1200.551736,20.520091,1.222273,43.175000,1.403547e+04,21.182909,1.216727,44.403455,...,5020.2014,9.7,60340.800000,05|07,-0.050,-0.166683,0.001,0.0,200,221
0 days 00:02:18,2.0,137.993885,1201.113462,20.557133,1.220663,43.513612,1.404957e+04,21.081939,1.220235,44.595745,...,5020.2014,9.7,60341.408163,05|07,-0.052,-0.165728,0.001,0.0,200,221
0 days 00:02:19,2.0,138.999369,1202.118929,20.635529,1.217696,43.665510,1.407482e+04,20.999480,1.225176,44.707304,...,5020.2014,9.7,60342.349057,05|07,-0.051,-0.165576,0.001,0.0,200,221
0 days 00:02:20,2.0,140.000394,1203.119943,20.715600,1.212442,43.540411,1.409996e+04,21.150800,1.218021,44.659474,...,5020.2014,9.7,60343.277228,05|07,-0.051,-0.165597,0.001,0.0,200,221
0 days 00:02:21,2.0,140.999270,1204.118814,20.637881,1.217366,43.537604,1.412506e+04,21.080851,1.222149,44.650535,...,5020.2014,9.7,60344.207547,05|07,-0.050,-0.165157,0.001,0.0,201,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 14:39:31,2.0,52770.997042,53833.132747,-0.083859,-0.017313,0.002424,1.355884e+06,-0.082424,-0.017576,0.002323,...,5110.7016,286.3,204054.030612,05|07,-0.043,-0.143247,0.001,0.0,-7,-8
0 days 14:39:32,2.0,52772.000473,53834.136150,-0.083960,-0.017560,0.002490,1.355884e+06,-0.082430,-0.017490,0.002280,...,5110.7016,286.3,204055.000000,05|07,-0.044,-0.144925,0.001,0.0,-7,-8
0 days 14:39:33,2.0,52773.003900,53835.139566,-0.083919,-0.017273,0.002444,1.355884e+06,-0.082646,-0.017525,0.002333,...,5110.7016,286.3,204055.979167,05|07,-0.042,-0.145139,0.001,0.0,-7,-8


In [ ]:
# Load KNMI and Solcast data
knmi_data = Weer_data_inladen.load_knmi_data('data_weer/KNMI_weerdata.txt')
solcast_data = Weer_data_inladen.load_solcast_data('data_weer/Solcast_data_2.csv')

Weer_data_inladen.plot_comparison(knmi_data, solcast_data, '2024-06-01', '2024-06-15')